In [4]:
import requests
import numpy as np
import plots as pl
import pandas as pd
import matplotlib.pyplot as plt 
import aplpy 
import os
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from IPython.core.display import HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from astropy.coordinates import ICRS, Galactic, FK4, FK5
from astropy import units as u
from astropy.coordinates import SkyCoord

/Users/134393/anaconda3/lib/python3.7/site-packages/mpl_toolkits/axes_grid/__init__.py:12: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  obj_type='module')


In [5]:
def get_image(source_id, big=None):
    os.system('rm /Users/134393/Work/MWATS_Full/mwats/SQL/movies/*.png')
    image_all = filtered_raw_data[filtered_raw_data.source_id == source_id].sort_values(by='jd').image
    image_list = list(image_all)
    for im in range(len(image_list)):
        plt.figure(figsize=[10,12])
        image = image_list[im]
        s_stats = stats[stats.source_id == source_id]
        grid = True
        if big:
           size=(1200, 1200)
           im_rad = 8.0
        else: 
           size=(1200, 1200)
           im_rad = 1.0
        dpi = 40
        figure = aplpy.FITSFigure('/Volumes/BEL27A-BKP01/MWATS/'+image, dimensions=[0,1], north=False, slices=[0,0], auto_refresh=False, dpi = dpi, figsize = (size[0] / float(dpi), size[1] / float(dpi)))
        ra = s_stats.ra.values[0]
        dec = s_stats.dec.values[0]
        circle_size = 1.0 # 1 degree circle
        #figure.recenter(ra, dec, radius=im_rad, width=2.0, height=2.0)
        figure.recenter(25.597096, -17.238552, radius=im_rad, width=2.0, height=2.0)
        #figure.recenter(ra, dec, circle_size) 
        if grid:
            try:
                figure.add_grid()
            except Exception as e:
                logging.warn("APLpy error adding grid to image: {0}".format(e))
        # Adjust the axis labels, otherwise they overlap
        # If the image is less than two arc minutes across, show seconds on the tick labels
    
        image_radius = circle_size
        if image_radius < 1.0 / 60.0 * 2:
                figure.tick_labels.set_xformat("hh:mm:ss")
                figure.tick_labels.set_yformat("dd:mm:ss")
        else:
                figure.tick_labels.set_xformat("hh:mm")
                figure.tick_labels.set_yformat("dd:mm")

        figure.tick_labels.set_xformat("dd:mm")
        figure.tick_labels.set_yformat("dd:mm")
        figure.show_grayscale(pmin=5.0, pmax=98, invert=False)
        #figure.add_colorbar()
        # Show locations of the neighbours
        #figure.show_rectangles(ra, dec, 0.1, 0.1, edgecolor='k', linewidth=2.0)
        figure.show_rectangles(25.597096, -17.238552, 0.1, 0.1, edgecolor='r', linewidth=2.0)
        n = neighbours[neighbours.source_id == source_id]
        for num in range(5):
            if num == 0:
               neigh = n.n_id1.values[0]
               nra  =  n.n_id1_ra.values[0]
               ndec =  n.n_id1_dec.values[0]
               colour = 'm'
            if num == 1:
               neigh = n.n_id2.values[0]
               nra  =  n.n_id2_ra.values[0]
               ndec =  n.n_id2_dec.values[0]
               colour = 'y'
            if num == 2:
               neigh = n.n_id3.values[0]
               nra  =  n.n_id3_ra.values[0]
               ndec =  n.n_id3_dec.values[0]
               colour = 'c'
            if num == 3:
               neigh = n.n_id4.values[0]
               nra  =  n.n_id4_ra.values[0]
               ndec =  n.n_id4_dec.values[0]
               colour = 'b'
            if num == 4:
               neigh = n.n_id5.values[0]
               nra  =  n.n_id5_ra.values[0]
               ndec =  n.n_id5_dec.values[0]
               colour = 'r'
            figure.show_ellipses(nra, ndec, 0.1, 0.1, edgecolor=colour, linewidth=2.0)
        figure.axis_labels.set_font(size = 30.0)
        figure.tick_labels.set_font(size = 30.0)
        figure.add_colorbar()
        figure.colorbar.set_font(size = 30.0)
        figure.colorbar.set_axis_label_font(size = 30.0)
        figure.colorbar.set_axis_label_text('Jy / beam')
        if im <= 9:
           path = 'movies/00'+str(im)
        elif im > 99:
           path = 'movies/'+str(im)
        else:
           path = 'movies/0'+str(im)
        figure.save(path+'.png')
        plt.close()
        figure.close()

In [6]:
# Load the data
stats = pd.read_feather('stats_table.fth')
filtered_raw_data = pd.read_feather('data.fth')
#filtered_raw_avg_data = pd.read_feather('avg_data.fth')
neighbours = pd.read_feather('neighbours.fth')

In [7]:
source_id = 160820
bigit=False
get_image(source_id, big=bigit)
os.system('ffmpeg -framerate 10 -i %03d.png -s:v 1280x720 -c:v libx264 -profile:v high -crf 1 -pix_fmt yuv420p /Users/134393/Work/MWATS_Full/mwats/SQL/movies/'+str(source_id)+'.mp4')

INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.659e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.811e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.640e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.774e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.612e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.731e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.622e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.756e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.526e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.607e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.646e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.787e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.629e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.746e-01 [aplpy.core]

/Users/134393/anaconda3/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


INFO: Auto-setting vmin to -2.625e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.742e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.727e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.887e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.632e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.718e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.740e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.906e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.654e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.801e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.740e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.937e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.639e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.762e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]

INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.497e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.479e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.471e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.441e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.451e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.419e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.500e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.492e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.537e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.549e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.463e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.436e-01 [aplpy.core]
INFO: Setting slices=[0, 0] [aplpy.core]
INFO: Auto-setting vmin to -2.449e-01 [aplpy.core]
INFO: Auto-setting vmax to  4.417e-01 [aplpy.core]

256

...

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

<Figure size 720x864 with 0 Axes>

In [8]:
os.chdir('/Users/134393/Work/MWATS_Full/mwats/SQL/movies/')
os.system('ffmpeg -framerate 10 -i %03d.png -s:v 1280x720 -c:v libx264 -profile:v high -crf 1 -pix_fmt yuv420p /Users/134393/Work/MWATS_Full/mwats/SQL/movies/'+str(source_id)+'.mp4')

0